In [1]:
import pandas as pd
import panel as pn
import numpy as np
from io import BytesIO
import random
import torch
import torch.nn as nn
import torch.optim as optim
from io import StringIO

In [2]:
css = ['https://cdn.datatables.net/1.10.24/css/jquery.dataTables.min.css',
       # Below: Needed for export buttons
       'https://cdn.datatables.net/buttons/1.7.0/css/buttons.dataTables.min.css'
]

css2 = '''
.bk.panel-widget {
  border: None;
  font-size: 20px;
}

.button .bk-btn{
  font-size:20px;
  font-family: NanumBarunGothic;
}

.widget-button .bk-btn {
  font-size:20px;
  font-family: NanumBarunGothic;
}

.table .tabulator {
  font-size: 20px;
}

'''



js = {
    '$': 'https://code.jquery.com/jquery-3.5.1.js',
    'DataTable': 'https://cdn.datatables.net/1.10.24/js/jquery.dataTables.min.js',
    # Below: Needed for export buttons
    'buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/dataTables.buttons.min.js',
    'jszip': 'https://cdnjs.cloudflare.com/ajax/libs/jszip/3.1.3/jszip.min.js',
    'pdfmake': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/pdfmake.min.js',
    'vfsfonts': 'https://cdnjs.cloudflare.com/ajax/libs/pdfmake/0.1.53/vfs_fonts.js',
    'html5buttons': 'https://cdn.datatables.net/buttons/1.7.0/js/buttons.html5.min.js',
}
pn.extension(sizing_mode='stretch_width',css_files=css, raw_css=[css2], js_files=js)

In [3]:
class MLPModel(nn.Module): # 원래조건 : (5,20) (20,15), (15,10), (10,1)
    def __init__(self): 
        super(MLPModel, self).__init__()
        self.linear1 = nn.Linear(6,24)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(24,18)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(18,12)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(12,1)

    def forward(self, x):
    # 인스턴스(샘플) x가 인풋으로 들어왔을 때 모델이 예측하는 y값을 리턴합니다.
        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x
    
model = MLPModel()
model.load_state_dict(torch.load('full-para_training_model.pth'))

<All keys matched successfully>

In [11]:
kr_test_data = pd.read_csv("eu_asia.csv")
kr_test_data2=kr_test_data.copy()

if 'Dissolved Cu' in kr_test_data :
    kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC", "Alkalinity"]] 
    kr_test_X = kr_test_data.iloc[:,:6]

    kr_test_input_x = torch.Tensor(kr_test_X.values)

    kr_test_reshape_NN = torch.reshape(model(kr_test_input_x), (-1,))
    a = kr_test_reshape_NN.detach().numpy()
    a = pd.DataFrame(a)
    a.columns=['BLM-based PNECs']
    a=pd.merge(kr_test_data,a,left_index=True,right_index=True)
    a['RCR']=a['BLM-based PNECs'] / kr_test_data2['Dissolved Cu']

    df=a
else:
    kr_test_data = kr_test_data[["pH", "Na", "Mg", "Ca", "DOC", "Alkalinity"]] 

    kr_test_X = kr_test_data.iloc[:,:6]

    kr_test_input_x = torch.Tensor(kr_test_X.values)

    kr_test_reshape_NN = torch.reshape(model(kr_test_input_x), (-1,))
    a = kr_test_reshape_NN.detach().numpy()
    a = pd.DataFrame(a)
    a.columns=['BLM-based PNECs']
    a=pd.merge(kr_test_data,a,left_index=True,right_index=True)

    df=a

In [12]:
df =df[['pH']]*2

In [ ]:
df2=df.copy()

In [ ]:
df=df[['BLM-based PNECs']]
df=df.rename(columns={'BLM-based PNECs':'eu'})

In [36]:
df

,eu
0,3.778861
1,17.369524
2,58.645630
3,7.348326
4,15.790835
...,...
281,52.289948
282,30.431541
283,13.201858
284,14.477950


In [38]:
dfx=pd.concat([df2,df],axis=1)
dfx

,pH,Na,Mg,Ca,DOC,Alkalinity,BLM-based PNECs,eu
0,7.9,4.18,1.48,8.07,1.2,23.0,3.778861,3.778861
1,7.6,95.33,6.94,47.14,4.4,40.0,17.369524,17.369524
2,7.6,2.71,2.27,12.98,10.9,61.0,58.645630,58.645630
3,7.6,3.03,2.32,12.55,2.0,33.0,7.348326,7.348326
4,7.5,40.93,5.66,37.00,3.7,33.0,15.790835,15.790835
...,...,...,...,...,...,...,...,...
281,7.5,59.64,5.11,18.53,8.3,62.1,52.289948,52.289948
282,7.5,36.36,5.83,16.22,5.3,40.9,30.431541,30.431541
283,8.4,11.24,3.01,15.78,4.0,41.7,13.201858,13.201858
284,7.9,12.29,2.70,15.33,4.0,35.9,14.477950,14.477950


In [43]:
if('eu' in dfx.columns):
    print("x")
else:
    print("y")

x


In [6]:
class MLPModel_B(nn.Module): # 원래조건 : (5,20) (20,15), (15,10), (10,1)
    def __init__(self): 
        super(MLPModel_B, self).__init__()
        self.linear1 = nn.Linear(5,20)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(20,15)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(15,10)
        self.relu3 = nn.ReLU()
        self.linear4 = nn.Linear(10,1)

    def forward(self, x):
    # 인스턴스(샘플) x가 인풋으로 들어왔을 때 모델이 예측하는 y값을 리턴합니다.
        x = self.linear1(x)
        x = self.sigmoid1(x)
        x = self.linear2(x)
        x = self.sigmoid2(x)
        x = self.linear3(x)
        x = self.relu3(x)
        x = self.linear4(x)
        return x

In [7]:
pH = 7.06
Na = 46.0863
Ca = 29.1364
Mg = 5.06331
Dissolved_organic_carbon = 5.8
Individual_dissolved_Cu =1
model_B = MLPModel_B()
model_B.load_state_dict(torch.load('no_alk_training_model_kor.pth'))     

list=[float(pH),float(Na),float(Mg),float(Ca),float(Dissolved_organic_carbon)]
data=np.array(list)
kr_test_input_x = torch.Tensor(data)
kr_test_reshape_NN = torch.reshape(model_B(kr_test_input_x), (-1,))
result_b_asia=kr_test_reshape_NN.item()


output_cu_asia_b_b=float(result_b_asia)
x_cu_asia_b=float(Individual_dissolved_Cu)/result_b_asia

In [8]:
output_cu_asia_b_b

29.233062744140625

In [9]:
x_cu_asia_b

0.03420784228982085